In [ ]:
#!pip install langchain
from langchain.text_splitter import CharacterTextSplitter
!python --version

In [ ]:
# import openai
# openai.Model.list()

Read OPENAI API KEY

In [4]:
from langchain.chat_models import ChatOpenAI
from kor import create_extraction_chain, Object, Text, Number, extract_from_documents

with open("API_KEY", "r", encoding="utf-8") as f:
    key = f.read()

ImportError: cannot import name 'BaseLanguageModel' from 'langchain.schema' (C:\Users\jessi\OneDrive\School\lab\Data Extraction from PDF\code\pdf-extraction\venv2\lib\site-packages\langchain\schema.py)

Generate LengChain
Instantiate a langchain LLM and create a chain.

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=1500,
    openai_api_key=key
)

create schema for some category of pdf file

In [ ]:
appeal = Object(
    id="appeal",
    description="上訴人相關資訊;例如:法定代理人,訴訟代理人,代表人...等等",
    attributes=[
        Text(id="agent", description="代理人", many=True),
        Text(id="appellant", description="上訴人", examples=[("111年度交字第35號\n原告　蔡宗甫\n訴訟代理人　林虹妤\n", "蔡宗甫")], many=True),
        Text(id="representative", description="代表人", many=True),
        Text(id="attorney", description="訴訟代理人", examples=[("原告　蔡宗甫\n訴訟代理人　林虹妤\n被告　臺北市交通事件裁決所\n", "林虹妤")], many=True),
        Text(id="reagent", description="複代理人", many=True),
        Text(id="legal_representative", description="法定代理人", many=True),
        Text(id="defender", description="辯護人", many=True),
        Text(id="assistant", description="輔佐人", many=True)
    ]
)

appealed = Object(
    id="appealed",
    description="被上訴人相關資訊;例如:法定代理人,訴訟代理人,代表人...等等",
    attributes=[
        Text(id="agent", description="代理人", many=True),
        Text(id="appellee", description="被上訴人", examples=[("訴訟代理人　林虹妤\n被告　臺北市交通事件裁決所\n代表人　蘇福智\n", "臺北市交通事件裁決所")], many=True),
        Text(id="representative", description="代表人", examples=[("被告　臺北市交通事件裁決所\n代表人　蘇福智\n訴訟代理人　黃郁軒\n", "蘇福智")], many=True),
        Text(id="attorney", description="訴訟代理人", examples=[("代表人　蘇福智\n訴訟代理人　黃郁軒\n上列當事人間交通裁決事件，原告不服被告中華民國111年1月11\n", "黃郁軒")], many=True),
        Text(id="reagent", description="複代理人", many=True),
        Text(id="legal_representative", description="法定代理人", many=True),
        Text(id="defender", description="辯護人", many=True),
        Text(id="assistant", description="輔佐人", many=True)
    ]
)

schema_part1_1 = Object(
    id="judgment_info_part1",
    description="擷取分析裁判書的上半部分屬性值",
    attributes=[
        Text(
            id="title",
            description="這份裁判書的標題",
            examples=[("臺灣士林地方法院行政訴訟判決\n111年度交字第35號\n", "臺灣士林地方法院行政訴訟判決")],
        ),
        Text(
            id="id",
            description="這份裁判書的編號",
            examples=[("臺灣士林地方法院行政訴訟判決\n111年度交字第35號\n原告　蔡宗甫\n", "111年度交字第35號")],
        ),
        appeal,
        appealed,
        Text(
            id="foreword",
            description="這份裁判書的前言",
            examples=[("訴訟代理人　黃郁軒\n上列當事人間交通裁決事件，原告不服被告中華民國111年1月11\n日北市裁催字第22-AC0000000號、第22-AC0000000號裁決，提起\n行政訴訟，本院判決如下：\n主文\n", "上列當事人間交通裁決事件，原告不服被告中華民國111年1月11日北市裁催字第22-AC0000000號、第22-AC0000000號裁決，提起行政訴訟，本院判決如下：")],
        ),
        Text(
            id="main_content",
            description="這份裁判書的主文",
            examples=[("行政訴訟，本院判決如下：\n主文\n原告之訴駁回。\n訴訟費用新臺幣參佰元由原告負擔。\n事實及理由\n", "原告之訴駁回。訴訟費用新臺幣參佰元由原告負擔。")],
        )
    ],
    many=True
)

In [ ]:
chain_part1 = create_extraction_chain(llm, schema_part1_1, encoder_or_encoder_class="JSON", input_formatter=None)
print(chain_part1.prompt.format_prompt("[user input]").to_string())


In [ ]:
pdf_text = "公務員懲戒委員會裁定\n" \
       "102年度清字第11347號\n" \
       "移送機關　內政部　設臺北市○○區○○路○號\n" \
       "代表人　徐國勇　住同上\n" \
       "被付懲戒人　陳仁維　臺東縣警察局警務員\n" \
       "上列被付懲戒人因違法失職案件，經內政部移送審理，本會裁定\n"\
       "如下：\n"\
       "主文\n"\
       "本件停止審議程序之議決撤銷。\n"
print(len(pdf_text))


In [ ]:
from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    document_extraction_results = chain_part1.predict_and_parse(text=pdf_text)
    print(f"Output: {document_extraction_results.get('data')}")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")
